In [1]:
import os
os.chdir(os.path.expanduser(".."))

In [2]:
from src.collect.collect import SDMXCollector
import pandas as pd
from io import StringIO

In [3]:
from sqlalchemy import create_engine, URL

In [ ]:
collector = SDMXCollector("sdmx.oecd.org/public", 
                          "rest", 
                          {"format": "csv"})

n_args = 8
flow_ref = ["OECD.SDD.TPS", "DSD_BOP@DF_BOP", ""]

sample = collector.get(flow_ref, n_args)

In [5]:
pd.read_csv(StringIO(sample))

,DATAFLOW,REF_AREA,COUNTERPART_AREA,MEASURE,ACCOUNTING_ENTRY,FS_ENTRY,FREQ,UNIT_MEASURE,ADJUSTMENT,TIME_PERIOD,OBS_VALUE,OBS_STATUS,UNIT_MULT,CURRENCY,DECIMALS
0,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),PRT,WXD,CA,B,T,Q,PT_B1GQ,Y,2023-Q3,1.997802,E,0,_Z,2
1,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),RUS,WXD,S,D,T,Q,USD_EXC,N,2024-Q3,23698.310000,E,6,_Z,2
2,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),AUS,WXD,FA_O_F,A,T,Q,XDC,N,2024-Q1,-8611.000000,A,6,AUD,2
3,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),AUS,WXD,FA_O_F,A,T,Q,XDC,N,2023-Q4,-10996.000000,A,6,AUD,2
4,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),AUS,WXD,FA_O_F,A,T,Q,XDC,N,2023-Q3,1498.000000,A,6,AUD,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21383,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),USA,WXD,S,C,T,Q,USD_EXC,Y,2024-Q3,275304.000000,E,6,_Z,2
21384,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),OECD,WXD,CA,B,T,Q,PT_B1GQ,Y,2023-Q3,-0.205740,E,0,_Z,2
21385,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),OECD,WXD,CA,B,T,Q,PT_B1GQ,Y,2024-Q2,-0.036451,E,0,_Z,2
21386,OECD.SDD.TPS:DSD_BOP@DF_BOP(1.0),PRT,WXD,CA,B,T,Q,PT_B1GQ,Y,2024-Q2,2.554835,E,0,_Z,2


In [5]:
n_args = 7
flow_ref = ["OECD.SDD.STES", "DSD_KEI@DF_KEI", "4.0"]

sample = collector.get(flow_ref, n_args)

In [6]:
pd.read_csv(StringIO(sample))

,DATAFLOW,REF_AREA,FREQ,MEASURE,UNIT_MEASURE,ACTIVITY,ADJUSTMENT,TRANSFORMATION,TIME_PERIOD,OBS_VALUE,OBS_STATUS,UNIT_MULT,DECIMALS,BASE_PER
0,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),SWE,M,PRVM,GR,F,Y,GY,2023-10,-9.353234,P,0,1,NaN
1,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),SWE,M,PRVM,GR,F,Y,GY,2023-11,-9.274194,P,0,1,NaN
2,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),AUS,Q,PRVM,GR,BTE,Y,G1,2024-Q1,0.316224,A,0,1,NaN
3,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),AUS,Q,PRVM,GR,BTE,Y,G1,2024-Q2,0.117852,A,0,1,NaN
4,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),AUS,Q,BCICP,PB,C,Y,_Z,2023-Q3,7.333333,A,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34905,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),SWE,M,PRVM,IX,F,Y,_Z,2024-02,113.766567,P,0,1,2015.0
34906,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),SWE,M,PRVM,IX,F,Y,_Z,2024-03,111.329628,P,0,1,2015.0
34907,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),SWE,M,PRVM,IX,BTE,Y,_Z,2024-07,114.569410,P,0,1,2015.0
34908,OECD.SDD.STES:DSD_KEI@DF_KEI(4.0),SWE,M,PRVM,IX,BTE,Y,_Z,2024-08,116.287951,P,0,1,2015.0


In [ ]:
collector = SDMXCollector("data-api.ecb.europa.eu", "service")
flow_ref = "EXR"

sample = collector.get(flow_ref)